In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT
import glob
import sys
import os


Welcome to JupyROOT 6.24/00


In [26]:
data_dir = '../build/data/'

# number of primaries for given run
# if not in this dict, assume files have 5e7 primaries
# (distributed between all threads)
primary_dict = {
    'simdata_20241231_121236': 1e7,
    'simdata_20241231_145727': 5e6
}

# dict to hold [file path -- thread counts]
data_files_found = {}

filenames = os.listdir(data_dir)

Nprimaries = 0. # progressively add to this as more files read

for fname in filenames:
    fbase = fname.split('_t')[0]
    idx = int(fname.split('_t')[1].split('.')[0])

    if fbase not in data_files_found:
        data_files_found[fbase] = []
    
    data_files_found[fbase].append(idx)

for fbase, idxs in data_files_found.items():
    if all(i in idxs for i in range(max(idxs)+1)):
        numfiles = len(idxs)
        if numfiles != 8:
            print(fbase, ': numfiles =', numfiles)
    else:
        print(fbase, ': not all found')
    if fbase in primary_dict:
        Nprimaries += primary_dict[fbase]
        #print('found', fbase)
    else:
        Nprimaries += 5e7

print(f'total {Nprimaries:.4g} primaries')

simdata_20241231_121236 : numfiles = 4
simdata_20241231_145727 : numfiles = 4
simdata_20241231_155127 : numfiles = 4
total 2.615e+09 primaries


In [21]:
for fbase in primary_dict:
    print(fbase, '\n', data_files_found[fbase])

simdata_20241231_121236 
 [0, 1, 2, 3]
simdata_20241231_145727 
 [0, 1, 2, 3]


In [24]:
Nprimaries/1e9

2.615

In [30]:
def dcsd_to_string(tree, k):
    tree.GetEntry(k)
    EventNum = getattr(tree, 'EventNum')
    TrkNum = getattr(tree, 'TrkNum')
    Edep = float(getattr(tree, 'Edep'))
    VolName = getattr(tree, 'VolName')
    PName = getattr(tree, 'PName')
    ProcName = getattr(tree, 'ProcName')
    ParentVol = getattr(tree, 'ParentVol')
    IsCapture = getattr(tree, 'IsCapture')

    s = f"""
    EventNum        = {EventNum}
    TrkNum          = {TrkNum}
    Edep            = {Edep} [MeV]
    VolName         = {VolName}
    PName           = {PName}
    ProcName        = {ProcName}
    ParentVol       = {ParentVol}
    IsCapture       = {IsCapture}
    """

    return s

In [33]:
Ncaptures = 0 # number of neutron captures in the lowmass detector
Nhits = 0 # total number of entries in trees

for file in filenames:
    tfile = ROOT.TFile.Open(data_dir + file, 'READ')
    tree = tfile.Get('tree')
    N = tree.GetEntries()
    Nhits += N
    n = tree.GetEntries('IsCapture')
    if n > 0:
        Ncaptures += n
        N = tree.GetEntries()
        for k in range(N):
            tree.GetEntry(k)
            if int(getattr(tree, 'IsCapture')) == 1:
                print(dcsd_to_string(tree, k))
        

    tfile.Close()

print(f'found {Nhits:.3g} hits total, {Ncaptures} capture events in lowmass')


    EventNum        = 13803313
    TrkNum          = 34
    Edep            = 0.000667428789657555 [MeV]
    VolName         = lowmass
    PName           = Si29
    ProcName        = nCapture
    ParentVol       = lowmass
    IsCapture       = 1
    

    EventNum        = 20715467
    TrkNum          = 56
    Edep            = 0.0006678201730234441 [MeV]
    VolName         = lowmass
    PName           = Si29
    ProcName        = nCapture
    ParentVol       = lowmass
    IsCapture       = 1
    

    EventNum        = 7557213
    TrkNum          = 29
    Edep            = 0.00066057350094328 [MeV]
    VolName         = lowmass
    PName           = Si29
    ProcName        = nCapture
    ParentVol       = lowmass
    IsCapture       = 1
    

    EventNum        = 7530268
    TrkNum          = 32
    Edep            = 0.0007788720028968919 [MeV]
    VolName         = lowmass
    PName           = Si29
    ProcName        = nCapture
    ParentVol       = lowmass
    IsCapture     

In [29]:
Ncaptures

6